## Basic workflow in langchain

1. Define the LLM wrapper 
2. Define a chat template (or prompt)
3. Create a chain using the prompt and the llm wrapper
4. Invoke the chain. The chain is what does all the work

In [1]:
from langchain_community.llms import Ollama

llm = Ollama(model="llama2")
llm.invoke("Do pigs fly?")

'\nPigs do not fly. Pigs are mammals and do not have the ability to fly. They are ground-dwelling animals and spend most of their time on the ground or in water. While some birds, such as penguins and seagulls, are able to fly, pigs are not capable of flight.'

In [2]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a world class story teller."),
    ("user", "{input}")
])

In [3]:
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

chain = prompt | llm | output_parser

In [4]:
story = chain.invoke(
    {"input": "Write a story in five lines about a pig happily flying in the sky."})
print(story)


With a grin and a wink,
Porcine Palmer soared on his wings,
Flying high above the ground,
Chortling with joy all around,
In the sky, he found his delight.


## Retrieval

1. Find a document source
2. Index doc into a vector store
3. Load doc embeddings
4. Create smaller chunks from large docs
5. Create a retrieval chain
6. Invoke

In [5]:
from langchain_community.document_loaders import WebBaseLoader
loader = WebBaseLoader("https://jujutsu-kaisen.fandom.com/wiki/Yuji_Itadori")

docs = loader.load()

In [6]:
from langchain_community.embeddings import OllamaEmbeddings

embeddings = OllamaEmbeddings()

In [7]:
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter()
documents = text_splitter.split_documents(docs)
vector = FAISS.from_documents(documents, embeddings)

In [8]:
from langchain.chains.combine_documents import create_stuff_documents_chain

prompt = ChatPromptTemplate.from_template("""Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}""")

document_chain = create_stuff_documents_chain(llm, prompt)

In [9]:
from langchain.chains import create_retrieval_chain

retriever = vector.as_retriever()
retrieval_chain = create_retrieval_chain(retriever, document_chain)

In [10]:
response = retrieval_chain.invoke(
    {"input": "What is Divergent Fist?"})
print(response["answer"])

Divergent Fist is a technique in the Jujutsu series that allows its user to create and control multiple versions of their body, each with different abilities and attributes. This technique is unique to Yuji Itadori and is one of his signature abilities as a Jujutsu sorcerer.

As a Divergent Fist user, Yuji can create up to three separate bodies, each with its own set of abilities and attributes. These bodies are known as "fists" and can be used independently or in combination with the main body. The fists are created through a process called "fusion," which allows Yuji to merge his own body parts with those of other beings, such as animals or objects.

Each fist has its own unique abilities and attributes, such as enhanced strength, speed, or durability. For example, the "Wild Fist" is able to transform into a powerful animal form, while the "Cloud Fist" is able to create and control clouds to use as a weapon or for transportation. The "Ice Fist" is able to generate and control ice to 

In [11]:
response = retrieval_chain.invoke(
    {"input": "How many times can Yuji use Black Flash in a row?"})
print(response["answer"])

According to the Jujutsu series, Yuji Itadori can use his Black Flash technique multiple times in a row, but there are some limitations and restrictions on how often he can use it.

In the manga, it is shown that Yuji can use Black Flash up to three times in a row before he needs to rest and recharge. This limit applies to both the manga and anime adaptations of the series.

However, it's worth noting that Yuji's Black Flash technique is considered a high-level innate technique, and its usage can have consequences on his body and mind. Therefore, he may need to take breaks and recover after prolonged use of this technique.

In summary, Yuji Itadori can use Black Flash up to three times in a row in the Jujutsu series, but he needs to rest and recharge afterward to avoid any adverse effects.


In [12]:
response = retrieval_chain.invoke(
    {"input": "What relationship does Yuji have with Sukuna?"})
print(response["answer"])


Answer: Yuji has a Binding Vow with Sukuna, which means that he is bound to the King of Curses through a contractual agreement. The terms of the agreement are that Yuji will allow Sukuna to take over his body for one full minute when Sukuna chants "Enchain," and Yuji will forget about the agreement after it is fulfilled. However, Yuji initially opposed the Binding Vow and made Sukuna promise that he wouldn't kill or hurt anyone during that minute, indicating a level of distrust or unease with the arrangement.


In [13]:
response = retrieval_chain.invoke(
    {"input": "Make a bullet point list of Yuiji's abilities."})
print(response["answer"])


Sure, here are Yuji's abilities and powers as listed in the context:

• Overall Skill Level: Yuji is one of Tokyo Jujutsu High's most powerful and talented students. Satoru Gojo places him on the same level of potential as Yuta Okkotsu and Kinji Hakari, the three students who he believes will surpass his strength.
• Immense Strength: Yuji is quite powerful even without the usage of cursed energy, as he can destroy walls with ease, throw a lead ball with enough force to bend a soccer goalpost, and even throw a car.
• Immense Speed and Reflexes: Yuji is shown to have great speed and can finish a 50-meter track within 3 seconds, implying he can run at speeds of 60.12km/h (37.3mph). He's also been shown to dodge the attacks of all five opposing sorcerers from the students from Kyoto Jujutsu High who tried to murder him.
• Immense Durability: Yuji's durability surpasses the average sorcerer. He was able to continuously take hits from Choso during their fight and even Piercing Blood, an att